# Text data transformations

#### Description:

This codebook covers how to perform common text data transformations.

#### Skill level:

- Intermediate

### Import the required libraries
-------------------------

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn as sns

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\buswedg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Create a corpus of text data
-------------------------

In [2]:
corpus = ['The sky is blue and beautiful.',
          'Love this blue and beautiful sky!',
          'The quick brown fox jumps over the lazy dog.',
          'The brown fox is quick and the blue dog is lazy!',
          'The sky is very blue and the sky is very beautiful today',
          'The dog is lazy but the brown fox is quick!']

labels = ['weather', 'weather', 'animals', 'animals', 'weather', 'animals']

corpus = np.array(corpus)

df_raw = pd.DataFrame({'document': corpus, 'category': labels})

In [3]:
df_raw

,document,category
0,The sky is blue and beautiful.,weather
1,Love this blue and beautiful sky!,weather
2,The quick brown fox jumps over the lazy dog.,animals
3,The brown fox is quick and the blue dog is lazy!,animals
4,The sky is very blue and the sky is very beaut...,weather
5,The dog is lazy but the brown fox is quick!,animals


### Normalize the corpus of text
-------------------------

In [4]:
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I)
    doc = doc.lower()
    doc = doc.strip()
    
    # tokenize document
    tokens = wpt.tokenize(doc)
    
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    
    return doc

normalize_corpus = np.vectorize(normalize_document)

In [5]:
norm_corpus = normalize_corpus(corpus)

norm_corpus

array(['sky blue beautiful', 'love blue beautiful sky',
       'quick brown fox jumps lazy dog', 'brown fox quick blue dog lazy',
       'sky blue sky beautiful today', 'dog lazy brown fox quick'],
      dtype='<U30')

### Add the normalized text to the original dataframe and inspect the data
-------------------------

In [6]:
df_norm = df_raw.copy()

df_norm['norm_doc'] = norm_corpus

In [7]:
df_norm.head()

,document,category,norm_doc
0,The sky is blue and beautiful.,weather,sky blue beautiful
1,Love this blue and beautiful sky!,weather,love blue beautiful sky
2,The quick brown fox jumps over the lazy dog.,animals,quick brown fox jumps lazy dog
3,The brown fox is quick and the blue dog is lazy!,animals,brown fox quick blue dog lazy
4,The sky is very blue and the sky is very beaut...,weather,sky blue sky beautiful today


### Create a vectorizor to label the existence of uni-grams in each document
-------------------------

In [8]:
cv = CountVectorizer(min_df=0., max_df=1.)

cv_matrix = cv.fit_transform(norm_corpus)
cv_matrix = cv_matrix.toarray()
cv_matrix

array([[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0],
       [0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 2, 1],
       [0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0]], dtype=int64)

### Add feature names and inspect the data
-------------------------

In [9]:
vocab = cv.get_feature_names()

df_bag_unigram = pd.DataFrame(cv_matrix, columns=vocab)

In [10]:
df_bag_unigram.head()

,beautiful,blue,brown,dog,fox,jumps,lazy,love,quick,sky,today
0,1,1,0,0,0,0,0,0,0,1,0
1,1,1,0,0,0,0,0,1,0,1,0
2,0,0,1,1,1,1,1,0,1,0,0
3,0,1,1,1,1,0,1,0,1,0,0
4,1,1,0,0,0,0,0,0,0,2,1


### Create a vectorizor to label the count of bi-grams in each document
-------------------------

In [11]:
bv = CountVectorizer(ngram_range=(2,2))

bv_matrix = bv.fit_transform(norm_corpus)
bv_matrix = bv_matrix.toarray()
bv_matrix

array([[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]], dtype=int64)

### Add feature names and inspect the data
-------------------------

In [12]:
vocab = bv.get_feature_names()

df_bag_bigram = pd.DataFrame(bv_matrix, columns=vocab)

In [13]:
df_bag_bigram.head()

,beautiful sky,beautiful today,blue beautiful,blue dog,blue sky,brown fox,dog lazy,fox jumps,fox quick,jumps lazy,lazy brown,lazy dog,love blue,quick blue,quick brown,sky beautiful,sky blue
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,1,0,1,0,1,0,1,0,0,1,0,0
3,0,0,0,1,0,1,1,0,1,0,0,0,0,1,0,0,0
4,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1


### Create a vectorizor to calculate the tf-idf measure in each document
-------------------------

In [14]:
tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)

tv_matrix = tv.fit_transform(norm_corpus)
tv_matrix = tv_matrix.toarray()

### Add feature names and inspect the data
-------------------------

In [15]:
vocab = tv.get_feature_names()

df_tfidf = pd.DataFrame(np.round(tv_matrix, 2), columns=vocab)

In [16]:
df_tfidf.head()

,beautiful,blue,brown,dog,fox,jumps,lazy,love,quick,sky,today
0,0.60,0.52,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.60,0.00
1,0.46,0.39,0.00,0.00,0.00,0.00,0.00,0.66,0.00,0.46,0.00
2,0.00,0.00,0.38,0.38,0.38,0.54,0.38,0.00,0.38,0.00,0.00
3,0.00,0.36,0.42,0.42,0.42,0.00,0.42,0.00,0.42,0.00,0.00
4,0.36,0.31,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.72,0.52
